<p style="text-align: center;">МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ
РОССИЙСКОЙ ФЕДЕРАЦИИ

<p style="text-align: center;">Федеральное государственное автономное
образовательное учреждение высшего образования
«Самарский национальный исследовательский университет
имени академика С. П. Королева»
(Самарский университет)</p>
<br>
<br>
<br>

<p style="text-align: center;">Институт информатики и кибернетики
    
<p style="text-align: center;">Факультет информатики
    
<p style="text-align: center;">Кафедра программных систем
    
 <br><br><br>   

<p style="text-align: center;">ОТЧЁТ

<p style="text-align: center;">по лабораторной работе № 3
<p style="text-align: center;">«Интеллектуальный анализ данных с библиотекой scikit-learn»
<p style="text-align: center;">по курсу «Интеллектуальный анализ данных и большие данные»

<p style="text-align: center;">
<br><br><br><br><br><br><br><br>







<p style="text-align: right;">Выполнил: <Фамилия И.О.>
<p style="text-align: right;">гр. 613X-020402D
<p style="text-align: right;">
<br><br><br><br><br><br><br><br><br>








<p style="text-align: center;">Самара 2025


# Интеллектуальный анализ данных с библиотекой scikit-learn


## Упражнение 1

Протестируйте примеры matplotlib в блокноте: скопируйте ссылку на исходный код ``Source Code`` на каждой странице и поместите ее в блокнот, используя магию ``%load``. Используйте следующие ссылки:

https://matplotlib.org/2.0.2/mpl_examples/lines_bars_and_markers/fill_demo.py

http://matplotlib.org/mpl_examples/pylab_examples/ellipse_collection.py

In [ ]:
%load https://matplotlib.org/2.0.2/mpl_examples/lines_bars_and_markers/fill_demo.py

In [ ]:
%load http://matplotlib.org/mpl_examples/pylab_examples/ellipse_collection.py

## Упражнение 2

1) Введите данные в массив NumPy из файла iris5.data. Найдите образцы с ошибками в параметрах и удалите их из набора.
2) Введите данные в DataFrame из файла iris5.data. Найдите образцы с ошибками в параметрах и удалите их из набора.

In [19]:
import numpy as np


In [ ]:
import pandas as pd
from pandas import Series, DataFrame



## Упражнение 3

Классификация: 
1) Примените KNeighborsClassifier к набору данных `iris`. Поиграйте с разными значениями `n_neighbors` и понаблюдайте, как меняются результаты обучения и теста.
2) Примените RandomForestClassifier к набору данных `iris`. Поиграйте с разными значениями `n_neighbors` и понаблюдайте, как меняются результаты обучения и теста.

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.neighbors import KNeighborsClassifier


## Упражнение 4

Сокращение размерности:
Сравните результаты применения PCA, Isomap и t-SNE на 5-классовом подмножестве набора данных цифр (``load_digits(5)``).

*Указание:*
*Для t-SNE руководствуйтесь следующим примером из scikit-learn:*
https://scikit-learn.org/stable/auto_examples/manifold/plot_t_sne_perplexity.html

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=5)

# ...

In [ ]:
from sklearn.manifold import TSNE


In [ ]:
from sklearn.manifold import Isomap


## Упражнение 5
Для набора данных (см. ячейку ниже) с неизотропными кластерами, на которых kmeans не работает, подберите алгоритм кластеризации, который хорошо выделяет очевидные кластеры.

<img src="images/set_3.png" width="400">

In [23]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=170, n_samples=600)
rng = np.random.RandomState(74)

# ...